# 深度学习计算

## 层和块

## 参数管理


## 延后初始化


## 自定义层


## 读写文件

## GPU

In [27]:
import torch
from torch import nn
from d2l import torch as d2l

In [28]:
!nvidia-smi

Mon Jul 22 17:03:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.79                 Driver Version: 531.79       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 L...  WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8                1W /  N/A|   1020MiB /  8188MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [29]:
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')
torch.cuda.device_count()

1

In [30]:

def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [31]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [32]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [33]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y.device

device(type='cpu')

In [34]:
Z = X.cuda(1) # 因为本机只有一块GPU 所以这块失败了
print(X)
print(Z)

Y + Z #如果需要计算两个数据相加 我们先要移到同一块gpu上

Z.cuda(1) is Z #假设变量Z已经存在于第二个GPU上。 如果我们还是调用Z.cuda(1)会发生什么？ 它将返回Z，而不会复制并分配新内存。


RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


根据经验，多个小操作比一个大操作糟糕得多。 此外，一次执行几个操作比代码中散布的许多单个操作要好得多。 

如果一个设备必须等待另一个设备才能执行其他操作， 那么这样的操作可能会阻塞。 这有点像排队订购咖啡，而不像通过电话预先订购： 当客人到店的时候，咖啡已经准备好了。

In [35]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())
net(X), net[0].weight.data.device

(tensor([[1.0170],
         [1.0170]], device='cuda:0', grad_fn=<AddmmBackward0>),
 device(type='cuda', index=0))